In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import cv2
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve
from tensorflow import keras
from tensorflow.keras.layers import (Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization, Rescaling, 
                                     Resizing, Dropout, RandomRotation, RandomContrast, RandomFlip)
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, CSVLogger, EarlyStopping
from tensorflow.keras.regularizers  import L2, L1

In [ ]:
print('Number of GPUs available: ', len(tf.config.experimental.list_physical_devices('GPU')))
print('Test is built with CUDA: ', tf.test.is_built_with_cuda())

In [ ]:
train_directory = './dataset/Emotions Dataset/Emotions Dataset/train'
val_directory = './dataset/Emotions Dataset/Emotions Dataset/test'

CLASS_NAMES = ['angry', 'happy', 'sad']

CONFIGURATION = {
    'BATCH_SIZE': 32,
    'IM_SIZE': 256,
    'LEARNING_RATE': 0.001,
    'N_EPOCHS': 20,
    'DROPOUT_RATE': 0.0,
    'REGULARIZATION_RATE': 0.0,
    'N_FILTERS': 6,
    'KERNEL_SIZE': 3,
    'N_STRIDES': 1,
    'POOL_SIZE': 2,
    'N_DENSE_1': 100,
    'N_DENSE_2': 10,
    'NUM_CLASSES': 3,
}

CONFIGURATION = {
    "BATCH_SIZE": 32,
    "IM_SIZE": 256,
    "LEARNING_RATE": 1e-3,
    "N_EPOCHS": 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 3,
    "PATCH_SIZE": 16,
    "PROJ_DIM": 768,
    "CLASS_NAMES": ["angry", "happy", "sad"],
}

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CLASS_NAMES,
    color_mode='rgb',
    batch_size=CONFIGURATION['BATCH_SIZE'],
    image_size=(CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE']),
    shuffle=True,
    seed=99,
)

In [ ]:
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=val_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CLASS_NAMES,
    color_mode='rgb',
    batch_size=CONFIGURATION['BATCH_SIZE'],
    image_size=(CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE']),
    shuffle=True,
    seed=99,
)

In [ ]:
val_dataset

In [ ]:
for i in val_dataset.take(1):
    print(i)

In [ ]:
plt.figure(figsize=(12, 12))

for images, labels in train_dataset.take(1):
    for i in range(15):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i]/255.0)
        plt.title(CLASS_NAMES[tf.argmax(labels[i], axis=0).numpy()])
        plt.axis('off')

In [ ]:
plt.figure(figsize=(12, 12))

for images, labels in val_dataset.take(1):
    for i in range(15):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i]/255.0)
        plt.title(CLASS_NAMES[tf.argmax(labels[i], axis=0).numpy()])
        plt.axis('off')

In [ ]:
augment_layers = tf.keras.Sequential([
    RandomRotation(factor=(-0.025, 0.025),),
    RandomFlip(mode='horizontal',),
    RandomContrast(factor=0.1,),
])

In [ ]:
def augment_layer(image, label):
    return augment_layers(image, training=True), label

In [ ]:
def box(lambda):
    r_x = tf.cast(tfp.distributions.Uniform(0, CONFIGURATION['IM_SIZE']).sample(1)[0], dtype=tf.int32)
    r_y = tf.cast(tfp.distributions.Uniform(0, CONFIGURATION['IM_SIZE']).sample(1)[0], dtype-tf.int32)

    r_w = tf.cast(CONFIGURATION['IM_SIZE'] * tf.math.sqrt(1 - lambda), dtype=tf.int32)
    r_h = tf.cast(CONFIGURATION['IM_SIZE'] * tf.math.sqrt(1 - lambda), dtype=tf.int32)

    r_x = tf.clip_by_value(r_x - r_w / 2, 0, CONFIGURATION['IM_SIZE'])
    r_y = tf.clip_by_value(r_y - r_h / 2, 0, CONFIGURATION['IM_SIZE'])

    x_b_r = tf.clip_by_value(r_x + r_w / 2, 0 CONFIGURATION['IM_SIZE'])
    y_b_r = tf.clip_by_value(r_y + r_h / 2, 0, CONFIGURATION['IM_SIZE'])

    r_w = x_b_r - r_x
    if (r_w == 0):
        r_w = 1

In [ ]:
training_dataset = (
    train_dataset
    .map(augment_layer, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
validation_dataset = (
    val_dataset.prefetch(tf.data.AUTOTUNE)
)

In [ ]:
plt.figure(figsize=(12, 12))

for images, labels in training_dataset.take(1):
    for i in range(15):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i]/255.0)
        plt.title(CLASS_NAMES[tf.argmax(labels[i], axis=0).numpy()])
        plt.axis('off')

In [ ]:
resize_rescale_layers = tf.keras.Sequential([
    Resizing(CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE']),
    Rescaling(1./255),
])

In [ ]:
lenet_model = tf.keras.Sequential([
    InputLayer(input_shape=(None, None, 3), ),

    # InputLayer(input_shape=(CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE'], 3)),
    # Rescaling(1./255, name='rescaling'),

    resize_rescale_layers,

    Conv2D(filters=CONFIGURATION['N_FILTERS'],
           kernel_size=CONFIGURATION['KERNEL_SIZE'],
           strides=CONFIGURATION['N_STRIDES'],
           padding='valid',
           activation='relu',
           kernel_regularizer=L2(CONFIGURATION['REGULARIZATION_RATE'])),
    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGURATION['POOL_SIZE'],
              strides=CONFIGURATION['N_STRIDES'] * 2),
    Dropout(rate=CONFIGURATION['DROPOUT_RATE']),

    Conv2D(filters=CONFIGURATION['N_FILTERS'],
           kernel_size=CONFIGURATION['KERNEL_SIZE'],
           strides=CONFIGURATION['N_STRIDES'],
           padding='valid',
           activation='relu',
           kernel_regularizer=L2(CONFIGURATION['REGULARIZATION_RATE'])),
    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGURATION['POOL_SIZE'],
              strides=CONFIGURATION['N_STRIDES'] * 2),

    Flatten(),

    Dense(CONFIGURATION['N_DENSE_1'],
          activation='relu',
          kernel_regularizer=L2(CONFIGURATION['REGULARIZATION_RATE'])),
    BatchNormalization(),
    Dropout(rate=CONFIGURATION['DROPOUT_RATE']),

    Dense(CONFIGURATION['N_DENSE_2'], 
          activation='relu',
          kernel_regularizer=L2(CONFIGURATION['REGULARIZATION_RATE'])),
    BatchNormalization(),

    Dense(CONFIGURATION['NUM_CLASSES'],
          activation='softmax'),
])

lenet_model.summary()

In [ ]:
loss_function = CategoricalCrossentropy(from_logits=False)

In [ ]:
metrics = [CategoricalAccuracy(name='accuracy'), TopKCategoricalAccuracy(k=2, name='top_k_accuracy')]

In [ ]:
lenet_model.compile(
    optimizer=Adam(learning_rate=CONFIGURATION['LEARNING_RATE']),
    loss=loss_function,
    metrics=metrics
)

In [ ]:
history = lenet_model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=CONFIGURATION['N_EPOCHS'],
    verbose=1
)

In [ ]:
lenet_model.save('./HumanEmotions.keras')

In [ ]:
lenet_model = tf.keras.models.load_model('./HumanEmotions.keras')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'val_accuracy'])
plt.show

In [ ]:
lenet_model.evaluate(validation_dataset)

In [ ]:
happy_image = cv2.imread('./dataset/Emotions Dataset/Emotions Dataset/test/happy/707302.jpg')
happy_im = tf.constant(happy_image, dtype=tf.float32)

happy_im = tf.expand_dims(happy_im, axis=0)

print(CLASS_NAMES[tf.argmax(lenet_model(happy_im), axis=-1).numpy()[0]])

In [ ]:
sad_image = cv2.imread('./dataset/Emotions Dataset/Emotions Dataset/test/sad/99675.jpg')
sad_im = tf.constant(sad_image, dtype=tf.float32)

sad_im = tf.expand_dims(sad_im, axis=0)

print(CLASS_NAMES[tf.argmax(lenet_model(sad_im), axis=-1).numpy()[0]])

In [ ]:
plt.figure(figsize=(12, 12))

for images, labels in val_dataset.take(1):
    for i in range(15):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i]/255.0)
        plt.title('True Label - : ' + CLASS_NAMES[tf.argmax(labels[i], axis=0).numpy()] + '\n' + 
                  'Predicted Label - : ' + CLASS_NAMES[tf.argmax(lenet_model(tf.expand_dims(images[i], axis=0)), axis=-1).numpy()[0]])
        plt.axis('off')

In [ ]:
neo_img = cv2.imread('./neo.jpg')
neo = tf.constant(neo_img, dtype=tf.float32)
neo = tf.expand_dims(neo, axis=0)
print(CLASS_NAMES[tf.argmax(lenet_model(neo), axis=-1).numpy()[0]])

In [ ]:
plt.imshow(cv2.cvtColor(neo_img, cv2.COLOR_BGR2RGB))
plt.title('True Label - : sad\n' + 
          'Predicted Label - : ' + CLASS_NAMES[tf.argmax(lenet_model(neo), axis=-1).numpy()[0]])

In [ ]:
try_img = cv2.imread('./try.jpg')
trythis = tf.constant(try_img, dtype=tf.float32)
trythis = tf.expand_dims(trythis, axis=0)
print(CLASS_NAMES[tf.argmax(lenet_model(trythis), axis=-1).numpy()[0]])

In [ ]:
plt.imshow(cv2.cvtColor(try_img, cv2.COLOR_BGR2RGB))
plt.title('True Label - : happy\n' + 
          'Predicted Label - : ' + CLASS_NAMES[tf.argmax(lenet_model(trythis), axis=-1).numpy()[0]])

In [ ]:
predicted = []
labels = []

for im, label in validation_dataset:
    predicted.append(lenet_model(im))
    labels.append(label.numpy())

In [ ]:
print(np.concatenate([np.argmax(labels[:-1], axis=-1).flatten(), np.argmax(labels[-1], axis=-1).flatten()]))
print(np.concatenate([np.argmax(predicted[:-1], axis=-1).flatten(), np.argmax(labels[-1], axis=-1).flatten()]))

In [ ]:
pred = np.concatenate([np.argmax(labels[:-1], axis=-1).flatten(), np.argmax(labels[-1], axis=-1).flatten()])
lab = np.concatenate([np.argmax(predicted[:-1], axis=-1).flatten(), np.argmax(labels[-1], axis=-1).flatten()])

In [ ]:
cm = confusion_matrix(lab, pred)
print(cm)
plt.figure(figsize=(8, 8))

sns.heatmap(cm, annot=True,)
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')